In [3]:
import asyncio
import time
import cv2
import nats
import numpy as np
from nats.aio.msg import Msg
import nest_asyncio

from generated.project.common.proto.Inference_pb2 import InferenceList
from project.common import profiler
from project.common.config_class.profiler import ProfilerConfig
from project.generated.project.common.proto.Image_pb2 import ImageMessage
from project.generated.project.common.proto.Inference_pb2 import Inference

nest_asyncio.apply()

def render_detections(frame: np.ndarray, inference: Inference):
    for i in range(0, len(inference.bounding_box), 4):
        box = inference.bounding_box[i:i+4]
        x1, y1, x2, y2 = map(int, box)
        
        cv2.rectangle(
            frame,
            (x1, y1),
            (x2, y2),
            color=(0, 255, 0),
            thickness=2
        )
        
        # Add label with confidence
        label = f"{inference.class_name}: {inference.confidence:.2f}"
        cv2.putText(
            frame,
            label,
            (x1, y1 - 10),
            cv2.FONT_HERSHEY_SIMPLEX,
            0.5,
            (0, 255, 0),
            2
        )

async def main():
    profiler.init_profiler(
        ProfilerConfig(
            {
                "activated": True,
                "output-file": "profiler.html",
                "profile-function": True,
                "time-function": True,
            }
        )
    )

    cap = cv2.VideoCapture(0)
    nt = await nats.connect("nats://localhost:4222")

    queue = asyncio.Queue()
    image_id_map = {}

    async def on_message(msg: Msg):
        await queue.put(InferenceList.FromString(msg.data))

    await nt.subscribe("recognition/image_output", cb=on_message)

    image_id = 0
    while True:
        image_id += 1

        ret, frame = cap.read()
        if not ret:
            continue

        _, compressed_image = cv2.imencode(".jpg", frame)

        msg = ImageMessage(
            image=compressed_image.tobytes(),
            camera_name="camera0",
            is_gray=False,
            id=image_id,
            height=frame.shape[0],
            width=frame.shape[1],
            timestamp=int(time.time() * 1000),
        )

        image_id_map[image_id] = frame

        await nt.publish("recognition/image_input", msg.SerializeToString())
        await nt.flush()

        if not queue.empty():
            inference = await queue.get()
            for inference in inference.inferences:
                # print(int(time.time() * 1000) - inference.timestamp, str(queue.qsize()))
                render_detections(image_id_map[image_id], inference)
                cv2.imshow("frame", image_id_map[image_id])
                cv2.waitKey(1)
                image_id_map.pop(image_id)

await main()

2024-11-17 18:33:52.992 Python[72072:17763056] WARNING: Secure coding is not enabled for restorable state! Enable secure coding by implementing NSApplicationDelegate.applicationSupportsSecureRestorableState: and returning YES.


CancelledError: 

nats: encountered error
Traceback (most recent call last):
  File "/Users/godbrigero/Documents/BLITZ/.venv/lib/python3.12/site-packages/nats/aio/client.py", line 2117, in _read_loop
    await self._ps.parse(b)
  File "/Users/godbrigero/Documents/BLITZ/.venv/lib/python3.12/site-packages/nats/protocol/parser.py", line 157, in parse
    await self.nc._process_pong()
  File "/Users/godbrigero/Documents/BLITZ/.venv/lib/python3.12/site-packages/nats/aio/client.py", line 1597, in _process_pong
    future.set_result(True)
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/asyncio/futures.py", line 260, in set_result
    raise exceptions.InvalidStateError(f'{self._state}: {self!r}')
asyncio.exceptions.InvalidStateError: CANCELLED: <Future cancelled>


: 